In [ ]:
import matplotlib.pyplot as plt
from pyniva import PUB_META, PUB_TSB, Vessel, token2header

plt.style.use("ggplot")

# Data access using pyniva

*Notebook contributed by Zofia Rudjord*

This is an example demonstrating how to get data from **nivacloud** using **pyniva** and plot them. The pyniva package is already installed on the JupyterHub and a common access token for all users is stored on the `shared` drive. Eventually, we may switch to personal/individual access tokens, but for now all users of the JupyterHub can use the same shared token, as illustrated in the code below.

Here is how you start:

In [ ]:
path_to_token = "/home/jovyan/shared/01_datasets/tokens/niva-service-account.json"
HEADER = token2header(path_to_token)

To check which Vessels have associated data:

In [ ]:
boat = [v for v in Vessel.list(PUB_META, header=HEADER)]
print(*[b.path for b in boat], sep="\n")

To see which data exists for Color Fantasy 

In [ ]:
platform_code = "FA"
first_n_measurements = 10  # = -1 will list all measurments
FA = [v for v in boat if v.path == platform_code][0]
measurements = FA.get_all_tseries(PUB_META, header=HEADER)
print(*[m.path for m in measurements][:first_n_measurements], sep="\n")

To get raw chlorophyll data for May-Jul 2019 ( note noqc = True setting in order to get all data without quality control)

In [ ]:
start_time = "2019-05-01T00:00:00"
end_time = "2019-07-31T00:00:00"
path = platform_code + "/ferrybox/CHLA_FLUORESCENCE/ADJUSTED"
for m in measurements:
    if m.path == path:
        data = m.get_tseries(
            PUB_TSB,
            header=HEADER,
            noqc=True,
            dt=0,
            start_time=start_time,
            end_time=end_time,
        )

print(data)

To plot the data

In [ ]:
plt.plot(data, color="k", label="raw data")
plt.gcf().autofmt_xdate()
plt.ylabel("chla_fluoresence_raw", fontsize=16)
plt.show()